1\. Write a function that converts number representation (bin<->dec<->hex)

In [1]:
#___Dictionary_for_hex_____
hex_dic = {"0":0, "1":1, "2":2, "3":3, 
           "4":4, "5":5, "6":6, "7":7, 
           "8":8, "9":9, "a":10, "b":11, 
           "c":12, "d":13, "e":14, "f":15 }


#___Simple_conversions_____
def int_to_bin(str_n):

    if str_n[1]=="x": #if hexadecimal, convert to decimal first
        str_n = hex_to_int(str_n)
    n = int(str_n)

    bin_n = ""
    while n!=0:
        if n%2==1:
            bin_n = "1" + bin_n
        else:
            bin_n = "0" + bin_n
        n = n//2      
    return ("0b"+bin_n)


def bin_to_int(n):
    n = n[2:]
    int_n = 0
    for i in range(len(n)):
        if n[i]=="1":
            int_n += 2**(len(n)-1-i)
    return str(int_n)


def int_to_hex(str_n):
    
    if str_n[1]=="b": #if binary, convert to decimal first
        str_n = bin_to_int(str_n)
    n = int(str_n)
    
    hex_n = ""
    while n!=0:
        for symbol in hex_dic:
            if hex_dic[symbol] == n%16:
                hex_n = symbol + hex_n
        n = n//16
    return ("0x"+hex_n)


def hex_to_int(n):
    n = n[2:]
    int_n = 0
    for i in range(len(n)):
        int_n += hex_dic[n[i]]*16**(len(n)-1-i)
    return str(int_n)


#___One_function_to_call_them_all______
def convert_to_everything(n):
   
    if n[1]=="b":
        print("Decimal representation of",n,"is", bin_to_int(n))
        print("Binary representation of",n,"is", n)
        print("Hexadecimal representation of",n,"is", int_to_hex(n))
    
    elif n[1]=="x":
        print("Decimal representation of",n,"is", hex_to_int(n))
        print("Binary representation of",n,"is", int_to_bin(n))
        print("Hexadecimal representation of",n,"is", n)
        
    else:
        print("Decimal representation of",n,"is", n)
        print("Binary representation of",n,"is", int_to_bin(n))
        print("Hexadecimal representation of",n,"is", int_to_hex(n))
        
#choosing the number and calling the converter
number = "0x12"
convert_to_everything(number)

Decimal representation of 0x12 is 18
Binary representation of 0x12 is 0b10010
Hexadecimal representation of 0x12 is 0x12


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [2]:
def float_number(string):
    sign = string[0]
    exponent_bin = string[1:9]
    mantissa_bin = string[9:]
    
    mantissa = 1
    for i in range(len(mantissa_bin)):
        if mantissa_bin[i] == "1":
            mantissa = mantissa + 2**(-(i+1))
    
    exponent = int(exponent_bin,2) - 127
    result = 1**(int(sign)) * mantissa * 2**exponent
    return (result)
    
    
number = "11110000111100001111000011110000" #-5.9654142E29
print(float_number(number))

5.965414233742897e+29


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [3]:
#find underflow limit
number = 1.
while number!=0:
    minimum = number
    number /= 2
print("Underflow limit is ", minimum)

#find overflow limit
number = 1.
while number!=float("inf"):
    maximum = number
    number *= 2
print("Overflow limit is ", maximum)

Underflow limit is  5e-324
Overflow limit is  8.98846567431158e+307


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [4]:
x = 1.
precision = 1
while 1:
    precision /= 2
    x_new = x + precision
    if x_new==x:
        break
    else:
        x = x_new

print("The machine precision is", precision)

The machine precision is 5.551115123125783e-17


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [5]:
from math import sqrt

def quad_sol(a,b,c):
    sol_1 = (-b+sqrt(b**2-4*a*c))/(2*a)
    sol_2 = (-b-sqrt(b**2-4*a*c))/(2*a)
    #print((-b+sqrt(b**2-4*a*c)),(2*a))
    return ([sol_1,sol_2])

def alternative_quad_sol(a,b,c):
    sol_1 = (b**2 - (b**2-4*a*c))/(2*a*(-b-sqrt(b**2-4*a*c)))
    sol_2 = (b**2 - (b**2-4*a*c))/(2*a*(-b+sqrt(b**2-4*a*c)))
     #print((b**2 - (b**2-4*a*c)),(2*a*(-b+sqrt(b**2-4*a*c))))
    return ([sol_1,sol_2])

def best_quad_sol(a,b,c):
    sol_1 = (-b)/(2*a) + sqrt(b**2-4*a*c)/(2*a)
    sol_2 = (-b)/(2*a) - sqrt(b**2-4*a*c)/(2*a)
    return ([sol_1,sol_2])
    
    
print('Results for a): ', quad_sol(0.001,1000,0.001))               #sol_1 viene calcolata male
print('Results for b): ', alternative_quad_sol(0.001,1000,0.001))   #sol_2 viene calcolata male
print('Results for c): ', best_quad_sol(0.001,1000,0.001))
# Utilizzando best_quad_sol si divide la frazione in due parti in modo che
# il numeratorenon si annulli (quasi) prima di fare la divisione


Results for a):  [-9.999894245993346e-07, -999999.999999]
Results for b):  [-1.00000761449437e-06, -1000018.1900864029]
Results for c):  [-1.00000761449337e-06, -999999.999999]


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [6]:
def f(x):
    return x*(x-1)

def derivate(sigma):
    x = 1
    der = (f(x+sigma) - f(x)) / sigma
    return der

for i in range(7):
    print("Derivate for sigma = 10^(-"+str(2*i+2)+") is equal to", derivate(10**(-2*i-2)))
    
#La derivata analitica è df/dx = 2x-1, che per x=1 vale 1.
#La sigma è però molto piccola e viene persa precisione quando viene salvata. 
#Questa differenza dal caso analitico viene poi amplificata usando la sigma come
#divisore per il calcolo della derivata (divisione per un numero molto piccolo)

#C'è un massimo dell'accuratezza per 10^(-8) (???)

Derivate for sigma = 10^(-2) is equal to 1.010000000000001
Derivate for sigma = 10^(-4) is equal to 1.0000999999998899
Derivate for sigma = 10^(-6) is equal to 1.0000009999177333
Derivate for sigma = 10^(-8) is equal to 1.0000000039225287
Derivate for sigma = 10^(-10) is equal to 1.000000082840371
Derivate for sigma = 10^(-12) is equal to 1.0000889005833413
Derivate for sigma = 10^(-14) is equal to 0.9992007221626509


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [6]:
def g(x):
    return sqrt(1-x**2)

def integrate(N):
    I = 0
    x = -1
    for i in range(N):
        I += 2/N * g(x+1/N) #considero il centro del bin
        x += 2/N
    return I

print(integrate(100))      #il risultato del programma è leggermente maggiore
#print(integrate(5000000))  #circa un secondo
#print(integrate(200000000)) #circa un minuto (risultato corretto fino all'ottava cifra dopo la virgola)

1.5712827762297947
